In [1]:
import numpy as np
import pandas as pd


In [7]:
df = pd.read_csv('USvideos.csv')

In [8]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [11]:
df_no_na = df.dropna()

In [15]:
import datetime
df_no_na['publish_time'] =  pd.to_datetime(df_no_na['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

/Users/apple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
df_no_na['trending_date'] =  pd.to_datetime(df_no_na['trending_date'], format='%y.%d.%m')

/Users/apple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
df_after_17 = df_no_na[df_no_na['publish_time'].dt.year >2017]

In [21]:
df_final = df_after_17[(df_after_17['trending_date'] >= '2018-01-01') & (df_after_17['trending_date'] <= '2018-05-01')]

In [25]:
ids = df_final.video_id.unique()

In [35]:
to_be_removed =[]
for curr_id in ids:
    cur = df_final[df_final['video_id']==curr_id]
    for index, row in cur.iterrows():
        if str(row['trending_date']) == '2018-05-01 00:00:00':
            to_be_removed+= [curr_id]
            break    

In [40]:
len(df_final)

21211

In [41]:
for curr_id in to_be_removed:
    df_final = df_final[df_final['video_id']!= curr_id]

In [42]:
len(df_final)

19801

In [69]:
df_final.to_csv("US_final.csv",index=False,sep="\\")

In [269]:
df_us = pd.read_csv("US_final.csv",sep="\\")

In [270]:
# add url link column
df_us["video_url"] = "https://www.youtube.com/watch?v="+df_us["video_id"]

In [271]:
#clean tags
df_tags_cleaned = df_us.copy()
new_tags = []
for index, row in df_us.iterrows():
    temp_tags= '|'.join([tag.strip().strip('"').strip('“').strip('”').strip() for tag in df_us.iloc[index]["tags"].split("|")])
    new_tags += [temp_tags]
df_tags_cleaned["tags"] = new_tags

In [272]:
df_tags_cleaned = df_tags_cleaned[df_tags_cleaned["tags"]!="[none]"]

In [ ]:
#add colunmns for category name and category cluster
# cate_map = {43 : "Entertainment",29 : "Nonprofits & Activism",28 : "Science & Technology",27 : "Education",26 : "Howto & Style",25 : "News & Politics",24 : "Entertainment",23 : "Comedy",22 : "People & Blogs",20 : "Gaming",19 : "Travel & Events",17 : "Sports",15 : "Pets & Animals",10 : "Music",2 : "Autos & Vehicles",1 : "Film & Animation"}

In [ ]:
df_tags_cleaned[df_tags_cleaned["category_id"]==28]

In [273]:
#clean date format
import datetime
import math
def days_taken_to_trend(start_time,end_time):
    def to_datetime(date_str):
        return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
#     start = to_datetime(df_tags_cleaned.iloc[0]["publish_time"])
    start = to_datetime(publish_time)
    end = to_datetime(trending_time)

    delta_days = (end-start if end>start else datetime.timedelta(days=1)-(start-end)).total_seconds()/(3600*24)
    return delta_days

In [274]:
#calculate temperature
#temp_raw_score = views*0.2+likes*0.3+cemment_count*0.5
#if not first day in trending : temp_score = (temp_raw_score of today - temp_raw_score of prev day)/ delta_day
#if first day in trending : temp_score = (temp_raw_score of today)/ delta_day since publish

unique_videos= df_tags_cleaned.video_id.unique()
df_with_temp = df_tags_cleaned.copy()
df_with_temp['temperature'] = 0
# print(unique_videos[:2])
for curr_vid in unique_videos:
    curr_df = df_with_temp[df_with_temp["video_id"]==curr_vid]
    i = 0
    prev_index = 0
    prev_raw_score = 0
    for index, row in curr_df.iterrows():
        curr_raw_score = row["views"]*0.2+row["likes"]*0.3+row["comment_count"]*0.5
        if i==0: #first day
            publish_time = row["publish_time"]
            trending_time = row["trending_date"]
            delta_days = float(days_taken_to_trend(publish_time,trending_time))
            curr_temp = curr_raw_score/delta_days
        else:
            start_time = curr_df.loc[prev_index]["trending_date"]
            end_time = curr_df.loc[index]["trending_date"]
            delta_days = math.ceil(days_taken_to_trend(start_time,end_time))
            curr_temp = (curr_raw_score-prev_raw_score)/delta_days
            if curr_raw_score < prev_raw_score:
                curr_temp = curr_raw_score / delta_days
            
        df_with_temp.at[index,"temperature"] = round(curr_temp,2)
        
        i+=1
        prev_index = index
        prev_raw_score = curr_raw_score
    

In [275]:
#scale temperature down to [0~100]
df_with_temp_scaled = df_with_temp.copy()
temp_range = math.log(max(df_with_temp["temperature"])-min(df_with_temp["temperature"]))
for index, row in df_with_temp.iterrows():
    try:
        df_with_temp_scaled.at[index,"temperature"] = math.log(df_with_temp.at[index,"temperature"])/temp_range*100
    except:
        df_with_temp_scaled.at[index,"temperature"] = math.log(0.001)/temp_range*100

In [278]:
df_with_temp_scaled.to_csv("US_final.csv",index=False,sep="\\")

In [276]:
df_with_temp_scaled# df_with_temp_scaled[df_with_temp_scaled["video_id"]=="0Juc2cL26mg"]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,video_url,temperature
0,vbiiik_T3Bo,2018-01-02 00:00:00,The End of the F**king World | Official Traile...,Netflix,24,2018-01-01 15:30:03,Netflix|Trailer|Netflix Original Series|Netfli...,1097679,31986,2527,3364,https://i.ytimg.com/vi/vbiiik_T3Bo/default.jpg,False,False,False,This isn’t your usual boy meets girl tale. Com...,https://www.youtube.com/watch?v=vbiiik_T3Bo,75
1,dbMtuJ1tZnc,2018-01-02 00:00:00,Karl Pilkington predicts Black Mirror (spoilers),matt,24,2018-01-01 01:05:59,ricky gervais|karl pilkington|black mirror|bla...,342214,6063,616,820,https://i.ytimg.com/vi/dbMtuJ1tZnc/default.jpg,False,False,False,After watching the finale of Black Mirror's 4t...,https://www.youtube.com/watch?v=dbMtuJ1tZnc,63
2,oFfMN6lPnlA,2018-01-02 00:00:00,LUKE SKYWALKER vs. KYLO REN 16 Bit,Mr Sunday Movies,24,2018-01-01 14:21:14,mr sunday movies|the weekly planet|luke skywal...,378862,19841,282,1584,https://i.ytimg.com/vi/oFfMN6lPnlA/default.jpg,False,False,False,Remember that bit in Star Wars The Last Jedi w...,https://www.youtube.com/watch?v=oFfMN6lPnlA,68
3,8V08Gu1DOgE,2018-01-02 00:00:00,Fifty Shades Freed - Mrs. Grey Will See You No...,Fifty Shades,24,2018-01-02 02:30:47,Fifty Shades of Grey|Official|Teaser|Teaser Tr...,244539,9858,655,651,https://i.ytimg.com/vi/8V08Gu1DOgE/default.jpg,False,False,False,#FiftyShadesFreed\n--\nFacebook: https://www.f...,https://www.youtube.com/watch?v=8V08Gu1DOgE,61
4,Bmn2STJUWJs,2018-01-02 00:00:00,Britney Spears - Toxic [Full HD] (New Year's E...,Piece Of Me Tour Planet Hollywood,10,2018-01-01 06:10:43,Britney Spears (icon)|Toxic [Full HD] (New Yea...,97689,1327,791,813,https://i.ytimg.com/vi/Bmn2STJUWJs/default.jpg,False,False,False,Get ‘The Essential Britney Spears’ on iTunes: ...,https://www.youtube.com/watch?v=Bmn2STJUWJs,57
5,bVU-MmJZFFA,2018-01-03 00:00:00,Justin Timberlake - INTRODUCING MAN OF THE WOODS,justintimberlakeVEVO,10,2018-01-02 15:00:04,Justin|Timberlake|Man|Of|The|Woods|Trailer|RCA...,725305,21236,1053,2861,https://i.ytimg.com/vi/bVU-MmJZFFA/default.jpg,False,False,False,Man Of The Woods \nFebruary 2nd \nNew Song Ja...,https://www.youtube.com/watch?v=bVU-MmJZFFA,72
6,8KgiaRPxRnQ,2018-01-03 00:00:00,Developer Update | Happy New Year | Overwatch,PlayOverwatch,20,2018-01-02 18:00:01,Developer Update|Happy New Year|Overwatch|Happ...,780084,39946,1089,10203,https://i.ytimg.com/vi/8KgiaRPxRnQ/default.jpg,False,False,False,From our family of heroes to yours: Happy New ...,https://www.youtube.com/watch?v=8KgiaRPxRnQ,75
7,8V08Gu1DOgE,2018-01-03 00:00:00,Fifty Shades Freed - Mrs. Grey Will See You No...,Fifty Shades,24,2018-01-02 02:30:47,Fifty Shades of Grey|Official|Teaser|Teaser Tr...,2130659,28119,4721,2933,https://i.ytimg.com/vi/8V08Gu1DOgE/default.jpg,False,False,False,#FiftyShadesFreed\n--\nFacebook: https://www.f...,https://www.youtube.com/watch?v=8V08Gu1DOgE,72
8,vUiE_0GPqKE,2018-01-03 00:00:00,"BØRNS, Lana Del Rey - God Save Our Young Blood...",BØRNSmusicVEVO,10,2018-01-02 17:00:06,BØRNS|Lana|Del|Rey|God|Save|Our|Young|Blood|In...,285261,29057,470,2764,https://i.ytimg.com/vi/vUiE_0GPqKE/default.jpg,False,False,False,“God Save Our Young Blood” featuring Lana Del ...,https://www.youtube.com/watch?v=vUiE_0GPqKE,69
9,72hlr-E7KA0,2018-01-03 00:00:00,How Airlines Price Flights,Wendover Productions,27,2018-01-02 16:30:26,ticket|pricing|airlines|airline|flights|flight...,373453,15829,267,1198,https://i.ytimg.com/vi/72hlr-E7KA0/default.jpg,False,False,False,Get 3 free meals from Blue Apron at http://coo...,https://www.youtube.com/watch?v=72hlr-E7KA0,70
